In [727]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import ast
import time
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint
from scipy import stats
from sklearn.cluster import AgglomerativeClustering
import folium
%autosave 60
%matplotlib inline

Autosaving every 60 seconds


In [974]:
#This JSON is an export of the versions table as a json. It includes all versions for our selected customers
with open("C:/Users/alexd/Downloads/cgr3.json") as data_file:
    data = json.load(data_file)

In [975]:
#Get our data into a dataframe
json_df = pd.DataFrame(data)

In [976]:
#Extract data from our json strings intothe column
json_df['object'] = json_df['object'].map(lambda x: dict(eval(x)))
json_df = json_df['object'].apply(pd.Series)

In [977]:
#Keep only the columns we need
json_df = json_df[['id','lat','lon','name','phone_number']]

In [978]:
#Get variables in the right type
json_df.lat = json_df.lat.astype('float') 
json_df.lon = json_df.lon.astype('float') 

In [979]:
#Drop null rows
json_df = json_df.dropna()

In [1112]:
#Drop all the rows with 0 values and trim outliers
json_df = json_df.loc[json_df['lon'] > 36] 
json_df = json_df.loc[json_df['lat'] < 0] 
#Filter out a test shop
json_df = json_df.loc[json_df['id'] != 19960] 
json_df = json_df.loc[json_df['id'] != 11417] 

In [981]:
#Drop duplicates
json_df = json_df.round({'lat': 5, 'lon': 5})
cols = ["id","lat","lon","name","phone_number"]
json_df = json_df[cols].loc[(json_df[cols].shift() != json_df[cols]).any(axis=1)]

In [982]:
#Drop values of the Nairobi warehouses and Nairobi Dam
json_df = json_df[(json_df.lat >= -1.28685) | (json_df.lat <= -1.2895) | (json_df.lon <= 36.7555) | (json_df.lon >= 36.76)]
json_df = json_df[(json_df.lat >= -1.3185) | (json_df.lat <= -1.3222) | (json_df.lon <= 36.898) | (json_df.lon >= 36.906)]
json_df = json_df[(json_df.lat >= -1.31879) | (json_df.lat <= -1.31975) | (json_df.lon <= 36.7967) | (json_df.lon >= 36.8035)]                                                                      

In [983]:
json_df.id = json_df.id.astype('int') 

In [984]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [985]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

In [986]:
def dbscan_reduce(df, epsilon, x='lon', y='lat'):
    start_time = time.time()
    # represent points consistently as (lat, lon) and convert to radians to fit using haversine metric
    coords = df.as_matrix(columns=['lat', 'lon'])   
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    
    # find the point in each cluster that is closest to its centroid
    centermost_points = clusters.map(get_centermost_point)

    # unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({x:lons, y:lats})
    rep_points.tail()
    
    # pull row from original data set where lat/lon match the lat/lon of each row of representative points
    rs = rep_points.apply(lambda row: df[(df[y]==row[y]) & (df[x]==row[x])].iloc[0], axis=1)
    
    clusterdf = pd.DataFrame(columns=['points'])
    clusterdf['points'] = pd.Series(clusters)
    clusterdf['points'] = clusterdf.points.str.len()
    
    rs['number_of_points'] = clusterdf['points']
    
    # all done, print outcome
    message = 'Clustered {:,} points down to {:,} points, for {:.2f}% compression in {:,.2f} seconds.'
    print(message.format(len(df), len(rs), 100*(1 - float(len(rs)) / len(df)), time.time()-start_time))    
    return rs

In [987]:
Uniqueids = json_df.id.unique()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in Uniqueids}

for key in DataFrameDict.keys():
    DataFrameDict[key] = json_df[:][json_df.id == key]

In [988]:
# epsilon=0.05km in radians
eps_rad = 0.05 / kms_per_radian

In [989]:
#Run the clustering algorithm and return the point nearest to the centroid for each cluster
#Then append it to our dataframe
df_clustered = pd.DataFrame()
for key in DataFrameDict:
    df_clustered = df_clustered.append(dbscan_reduce(DataFrameDict[key], epsilon=eps_rad))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Number of clusters: 45
Clustered 56 points down to 45 points, for 19.64% compression in 0.24 seconds.
Number of clusters: 22
Clustered 34 points down to 22 points, for 35.29% compression in 0.09 seconds.
Number of clusters: 11
Clustered 12 points down to 11 points, for 8.33% compression in 0.04 seconds.
Number of clusters: 55


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Clustered 78 points down to 55 points, for 29.49% compression in 0.17 seconds.
Number of clusters: 117


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


Number of clusters: 13
Clustered 13 points down to 13 points, for 0.00% compression in 0.06 seconds.
Number of clusters: 1
Clustered 1 points down to 1 points, for 0.00% compression in 0.02 seconds.
Number of clusters: 6
Clustered 6 points down to 6 points, for 0.00% compression in 0.04 seconds.
Number of clusters: 9
Clustered 9 points down to 9 points, for 0.00% compression in 0.04 seconds.
Number of clusters: 9
Clustered 9 points down to 9 points, for 0.00% compression in 0.05 seconds.
Number of clusters: 5
Clustered 5 points down to 5 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 5
Clustered 5 points down to 5 points, for 0.00% compression in 0.04 seconds.
Number of clusters: 9
Clustered 10 points down to 9 points, for 10.00% compression in 0.04 seconds.
Number of clusters: 2
Clustered 2 points down to 2 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 8
Clustered 9 points down to 8 points, for 11.11% compression in 0.04 seconds.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

Number of clusters: 1
Clustered 1 points down to 1 points, for 0.00% compression in 0.02 seconds.
Number of clusters: 1
Clustered 1 points down to 1 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 5
Clustered 5 points down to 5 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 2
Clustered 2 points down to 2 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 3
Clustered 3 points down to 3 points, for 0.00% compression in 0.02 seconds.
Number of clusters: 4
Clustered 4 points down to 4 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 3
Clustered 3 points down to 3 points, for 0.00% compression in 0.03 seconds.
Number of clusters: 4
Clustered 4 points down to 4 points, for 0.00% compression in 0.03 seconds.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [990]:
idx = df_clustered.groupby(['id'])['number_of_points'].transform(max) == df_clustered['number_of_points']

In [991]:
df_clustered2 = df_clustered[idx]

In [1111]:
#Save a csv of our locations
df_clustered2.to_csv("C:/Users/alexd/Downloads/cgr_assignments.csv")

In [994]:
df_clustered2['row_order'] = df_clustered2.index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [995]:
#If there are no clusters, take row 0.
df_clustered2 = df_clustered2[(df_clustered2.number_of_points > 1) | (df_clustered2.row_order == 0)]

In [996]:
#For competing clusters, take the earlier data point.
df_clustered2 = df_clustered2.groupby('id').first().reset_index()

In [1074]:
coords = df_clustered2.as_matrix(columns=['lat', 'lon'])
cluster = AgglomerativeClustering(n_clusters=37, affinity='euclidean', linkage='complete')
clustergroups = cluster.fit_predict(coords)
df_clustered2['clusters'] = clustergroups

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [1109]:
#This creates a map with the customers/assignments. I created the color assignments using trial and error.
locations = df_clustered2[['lat', 'lon']]
locationlist = locations.values.tolist()
colordict = {0: 'darkblue', 1: 'green', 2: 'red', 3: 'cadetblue', 4: 'darkred', 5: 'red', 6: 'lightgreen', 7: 'orange', 8: 'red', 9: 'green', 10: 'purple', 11: 'red', 12: 'cadetblue', 13: 'darkblue', 14: 'white', 15: 'purple', 16: 'darkblue', 17: 'orange', 18: 'darkblue', 19: 'green', 20: 'red', 21: 'darkblue', 22: 'red', 23: 'white', 24: 'red', 25: 'white', 26: 'green', 27: 'white', 28: 'darkblue', 29: 'orange', 30: 'green', 31: 'purple', 32: 'darkblue', 33: 'orange', 34: 'darkblue' , 35: 'green', 36: 'darkblue', 37: 'beige'}
map = folium.Map(location=[df_clustered2['lat'].mean(), df_clustered2['lon'].mean()], zoom_start=12)
for lat, lon, shop, cluster, name, phone in zip(df_clustered2['lat'], df_clustered2['lon'], df_clustered2['id'], df_clustered2['clusters'], df_clustered2['name'], df_clustered2['phone_number']):
    folium.CircleMarker([lat, lon], popup='Name: ' + str(name).capitalize() + '  Phone: ' + str(phone) + '  Cluster: ' + str(cluster) + '  ID: ' + str(shop), color=colordict[cluster], fill=True).add_to(map)
map

In [1116]:
#After reviewing and manually reassigning certain customers in excel, here is the final result.
df = pd.read_csv("C:/Users/alexd/Downloads/cgr_assignments.csv", header=0)

In [1120]:
locations = df[['lat', 'lon']]
locationlist = locations.values.tolist()
map = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
for lat, lon, shop, name, color in zip(df['lat'], df['lon'], df['id'], df['name'], df['colors']):
    folium.CircleMarker([lat, lon], popup='Name: ' + str(name).capitalize() + '  Region: ' + str(color).capitalize() + '  ID: ' + str(shop), color=(color), fill=True).add_to(map)
map

In [1121]:
map.save("C:/Users/alexd/Downloads/NW_map.html")